<a href="https://colab.research.google.com/github/bok-h22/TIL-202303/blob/master/BPE_RNN_%ED%9D%AC%EC%A4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tokenizers
#!pip install sentencepiece
#!pip install focal_loss
#!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
device = torch.device("cuda:0")
n_devices = torch.cuda.device_count()
print(n_devices)
for i in range(n_devices):
    print(torch.cuda.get_device_name(i))
torch.cuda.empty_cache()

1
Tesla T4


In [ ]:
import pandas as pd
import tensorflow as tf
import sentencepiece as spm

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# input/rename
data = pd.read_csv('/content/drive/MyDrive/한스펠 돌린 최종본.csv', encoding = 'utf-8')

data= data.drop(data[data['cleand_up'].isna() == True].index)
data= data.drop(data[data['cleand_down'].isna() == True].index)
data = pd.concat([data, pd.get_dummies(data['corp'])], axis = 1)
#data['label'] = data['label'].astype('str')

In [ ]:
# train/test split
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data.drop(['Unnamed: 0', 'corp', 'stock_code', 'year', 'sector', 'rank'], axis = 1), test_size=0.25, stratify=data['label'].to_list())
train_data, validation_data = train_test_split(train_data, test_size=0.2, stratify=train_data['label'].to_list())

In [ ]:
with open('test_review.txt', 'a', encoding='utf8') as f:
    f.write('\n'.join(train_data['cleand_up']))
    f.write('\n'.join(train_data['cleand_down']))
    f.write('\n'.join(test_data['cleand_up']))
    f.write('\n'.join(test_data['cleand_down']))

In [ ]:
# corp dummiies
corp_train = np.array(train_data.drop(['cleand_up', 'cleand_down', 'label'], axis = 1))
corp_valid = np.array(validation_data.drop(['cleand_up', 'cleand_down', 'label'], axis = 1))
corp_test = np.array(test_data.drop(['cleand_up', 'cleand_down', 'label'], axis = 1))

In [ ]:
from tokenizers import SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer()
tokenizer.train('test_review.txt', vocab_size=5000, min_frequency=3)

# train
train_data['sp_up_train'] = train_data['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
train_data['sp_down_train'] = train_data['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)
train_data['re_up_train'] = train_data['sp_up_train'].apply(lambda x: tokenizer.decode(x))
train_data['re_down_train'] = train_data['sp_down_train'].apply(lambda x: tokenizer.decode(x))

#valid
validation_data['sp_up_valid'] = validation_data['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
validation_data['sp_down_valid'] = validation_data['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)
validation_data['re_up_valid'] = validation_data['sp_up_valid'].apply(lambda x: tokenizer.decode(x))
validation_data['re_down_valid'] = validation_data['sp_down_valid'].apply(lambda x: tokenizer.decode(x))

# test
test_data['sp_up_test'] = test_data['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
test_data['sp_down_test'] = test_data['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)
test_data['re_up_test'] = test_data['sp_up_test'].apply(lambda x: tokenizer.decode(x))
test_data['re_down_test'] = test_data['sp_down_test'].apply(lambda x: tokenizer.decode(x))

In [ ]:
# 토크나이즈
tokenized_train = train_data['sp_down_train']
tokenized_valid = validation_data['sp_down_valid']
tokenized_test = test_data['sp_down_test']

In [ ]:
# 데이터 기술통계량
train_cnt = pd.DataFrame([ len(tokenized_train.iloc[i]) for i in range(len(tokenized_train)) ], columns = ['train'])
valid_cnt = pd.DataFrame([ len(tokenized_valid.iloc[i]) for i in range(len(tokenized_valid)) ], columns = ['valid'])
test_cnt = pd.DataFrame([ len(tokenized_test.iloc[i]) for i in range(len(tokenized_test)) ], columns = ['test'])

describe_1 = train_cnt.describe().values.tolist()
describe_2 = valid_cnt.describe().values.tolist()
describe_3 = test_cnt.describe().values.tolist()

pd.DataFrame([describe_1, describe_2, describe_3], columns = train_cnt.describe().index)

,count,mean,std,min,25%,50%,75%,max
0,[18395.0],[23.012449035063877],[20.25164636927158],[4.0],[14.0],[17.0],[24.0],[542.0]
1,[4599.0],[22.93889976081757],[20.10819922636494],[5.0],[14.0],[17.0],[25.0],[428.0]
2,[7665.0],[22.64187866927593],[18.05778952833602],[3.0],[14.0],[17.0],[24.0],[399.0]


In [ ]:
# Tokenize
max_length = 50

# train
train_sequences = np.concatenate([pad_sequences(tokenized_train, maxlen=max_length, padding='post', truncating='post'), corp_train], axis = 1)
labels_train = np.array(train_data['label'])

# valid
valid_sequences = np.concatenate([pad_sequences(tokenized_valid, maxlen=max_length, padding='post', truncating='post'), corp_valid], axis = 1)
labels_valid = np.array(validation_data['label'])

# test
test_sequences = np.concatenate([pad_sequences(tokenized_test, maxlen=max_length, padding='post', truncating='post'), corp_test], axis = 1)
labels_test = np.array(test_data['label'])

In [ ]:
test_sequences

array([[1052, 1622, 1808, ...,    0,    0,    0],
       [2077, 1966, 1019, ...,    0,    0,    0],
       [1018, 2204,  694, ...,    0,    0,    0],
       ...,
       [1042, 1157, 1213, ...,    0,    0,    0],
       [1782,  695, 1276, ...,    0,    0,    0],
       [2540,  737, 3232, ...,    0,    0,    0]], dtype=int32)

# 우리의 모델

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Dense, Dropout, BatchNormalization
from focal_loss import SparseCategoricalFocalLoss
from tensorflow.keras.models import Model

# Define input layer
input_layer = Input(shape=(max_length+len(data['corp'].unique()),))

# Embedding layer
embedding_layer = Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=512, input_length=max_length)(input_layer)

# LSTM layers
gru_1 = GRU(256, activation='tanh', return_sequences=True)(embedding_layer)
gru_2 = GRU(128, activation='tanh')(gru_1)

# Dropout and BatchNormalization layers
dropout_1 = Dropout(0.1)(gru_2)
batchnorm_1 = BatchNormalization()(dropout_1)
dense_1 = Dense(64, activation='relu')(batchnorm_1)

dropout_2 = Dropout(0.1)(dense_1)
batchnorm_2 = BatchNormalization()(dropout_2)
dense_2 = Dense(32, activation='relu')(batchnorm_2)

# Output layer
output_layer = Dense(9, activation='softmax')(dense_2)

# Create the functional model
model = Model(inputs=input_layer, outputs=output_layer)

# Create an optimizer with a custom learning rate
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Focal Loss
loss_func = SparseCategoricalFocalLoss(gamma=2)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 226)]             0         
                                                                 
 embedding_8 (Embedding)     (None, 226, 512)          2560000   
                                                                 
 gru_16 (GRU)                (None, 226, 256)          591360    
                                                                 
 gru_17 (GRU)                (None, 128)               148224    
                                                                 
 dropout_16 (Dropout)        (None, 128)               0         
                                                                 
 batch_normalization_16 (Bat  (None, 128)              512       
 chNormalization)                                                
                                                           

In [ ]:
history = model.fit(train_sequences,
                    labels_train,
                    validation_data=(valid_sequences, labels_valid),
                    verbose= 1,
                    epochs=100,
                    batch_size=64)

Epoch 1/100
288/288 [==============================] - 47s 144ms/step - loss: 2.0377 - accuracy: 0.1024 - val_loss: 1.7076 - val_accuracy: 0.1455
Epoch 2/100
288/288 [==============================] - 23s 80ms/step - loss: 1.7797 - accuracy: 0.1619 - val_loss: 1.5786 - val_accuracy: 0.1498
Epoch 3/100
288/288 [==============================] - 20s 68ms/step - loss: 1.6724 - accuracy: 0.1927 - val_loss: 1.6768 - val_accuracy: 0.2377
Epoch 4/100
288/288 [==============================] - 19s 64ms/step - loss: 1.6044 - accuracy: 0.2083 - val_loss: 1.5974 - val_accuracy: 0.1746
Epoch 5/100
288/288 [==============================] - 19s 66ms/step - loss: 1.5426 - accuracy: 0.2161 - val_loss: 1.5349 - val_accuracy: 0.1894
Epoch 6/100
288/288 [==============================] - 18s 64ms/step - loss: 1.4566 - accuracy: 0.2431 - val_loss: 1.3486 - val_accuracy: 0.2959
Epoch 7/100
288/288 [==============================] - 18s 63ms/step - loss: 1.3727 - accuracy: 0.2830 - val_loss: 1.2910 - val_a

In [ ]:
predict_test = model.predict(test_sequences)

predicted_classes = np.argmax(predict_test, axis=1)

240/240 [==============================] - 5s 17ms/step


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# 예측 결과(predicted_labels)와 실제 레이블(true_labels) 비교
accuracy = accuracy_score(labels_test, predicted_classes)

f1_score = f1_score(labels_test, predicted_classes, average = 'weighted')


print(f'accuracy : {accuracy:.3f}\nF1-score : {f1_score:.3f}')

accuracy : 0.841
F1-score : 0.840


In [ ]:
from keras.models import load_model

# 모델 저장하기
model.save('./20230725_gru_best_up.h5')

# Context 벡터 뽑아내기

In [ ]:
raw_up = data['cleand_up']
raw_down = data['cleand_down']

# corp dummiies
corp_total = np.array(data.drop(['Unnamed: 0', 'corp', 'stock_code', 'cleand_up', 'cleand_down', 'year', 'sector', 'rank', 'label'], axis = 1))
corp_total

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/20230725_gru_best_up.h5')

context_model = tf.keras.Model(inputs=model.input, outputs=model.layers[''].output)
input_data = train_sequences

context_vector = context_model.predict(input_data)

  3/575 [..............................] - ETA: 21s

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


575/575 [==============================] - 20s 34ms/step


# 재무데이터 합친 모델

In [ ]:
finalcial_df = pd.read_csv('/content/drive/MyDrive/final_a_df.csv', encoding = 'cp949')

finalcial_df = finalcial_df[['corp', 'stock_code', 'year', 'rank',
       'ebitda_margin', 'ebitda_to_interest_expense', 'debt_ratio',
       'dependence_on_net_borrowings', 'net_borrowings_to_ebitda', 'revenue',
       'cogs', 'selling_general_administrative_expenses', 'ebit',
       'ebit_margin', 'ebitda_to_sales_revenue', 'total_assets',
       'return_on_assets', 'ebitda', 'financial_expenses', 'corporate_tax',
       'operating_cash_flow', 'free_cash_flow', 'total_liabilities',
       'total_equity', 'total_borrowings', 'net_borrowings',
       'borrowing_dependency', 'total_borrowings_to_ebitda',
       'debt_to_net_income_ratio', 'total_assets_leverage',
       'current_liabilities', 'working_capital', 'current_liabilities_ratio',
       'quick_assets', 'quick_ratio', 'cash_and_cash_equivalents',
       'short_term_borrowings', 'days_sales_outstanding',
       'average_accounts_receivable_per_sales_turnover',
       'market_capitalization', 'minimum_wage', 'us_kor_exchange_avg',
       'ppi_year', 'kor_usa_ir_diff', 'kr_standard_yield', 'count', 'rating',
       'paywellfare', 'worklifebal', 'culture', 'opportunity', 'manager',
       'recommend', 'ceo', 'potential', 'crb_index_avg']]
text_raw = data[['corp', 'stock_code', 'year', 'rank', 'label', 'cleand_up', 'cleand_down']]

total_raw = text_raw.merge(finalcial_df, on = ['corp', 'stock_code', 'year', 'rank'], how = 'inner')

In [ ]:
# train/test split
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data.drop(['Unnamed: 0', 'corp', 'stock_code', 'year', 'sector', 'rank'], axis = 1), test_size=0.25, stratify=data['label'].to_list())
train_data, validation_data = train_test_split(train_data, test_size=0.2, stratify=train_data['label'].to_list())

In [ ]:
# corp dummiies
corp_train = np.array(train_data.drop(['cleand_up', 'cleand_down', 'label'], axis = 1))
corp_valid = np.array(validation_data.drop(['cleand_up', 'cleand_down', 'label'], axis = 1))
corp_test = np.array(test_data.drop(['cleand_up', 'cleand_down', 'label'], axis = 1))

In [ ]:
from tokenizers import SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer()
tokenizer.train('test_review.txt', vocab_size=5000, min_frequency=3)

# train
train_data['sp_up_train'] = train_data['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
train_data['sp_down_train'] = train_data['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)
train_data['re_up_train'] = train_data['sp_up_train'].apply(lambda x: tokenizer.decode(x))
train_data['re_down_train'] = train_data['sp_down_train'].apply(lambda x: tokenizer.decode(x))

#valid
validation_data['sp_up_valid'] = validation_data['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
validation_data['sp_down_valid'] = validation_data['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)
validation_data['re_up_valid'] = validation_data['sp_up_valid'].apply(lambda x: tokenizer.decode(x))
validation_data['re_down_valid'] = validation_data['sp_down_valid'].apply(lambda x: tokenizer.decode(x))

# test
test_data['sp_up_test'] = test_data['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
test_data['sp_down_test'] = test_data['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)
test_data['re_up_test'] = test_data['sp_up_test'].apply(lambda x: tokenizer.decode(x))
test_data['re_down_test'] = test_data['sp_down_test'].apply(lambda x: tokenizer.decode(x))

In [ ]:
# 토크나이즈
tokenized_train = train_data['sp_down_train']
tokenized_valid = validation_data['sp_down_valid']
tokenized_test = test_data['sp_down_test']

In [ ]:
# 데이터 기술통계량
train_cnt = pd.DataFrame([ len(tokenized_train.iloc[i]) for i in range(len(tokenized_train)) ], columns = ['train'])
valid_cnt = pd.DataFrame([ len(tokenized_valid.iloc[i]) for i in range(len(tokenized_valid)) ], columns = ['valid'])
test_cnt = pd.DataFrame([ len(tokenized_test.iloc[i]) for i in range(len(tokenized_test)) ], columns = ['test'])

describe_1 = train_cnt.describe().values.tolist()
describe_2 = valid_cnt.describe().values.tolist()
describe_3 = test_cnt.describe().values.tolist()

pd.DataFrame([describe_1, describe_2, describe_3], columns = train_cnt.describe().index)

In [ ]:
# Tokenize
max_length = 50

# train
train_sequences = np.concatenate([pad_sequences(tokenized_train, maxlen=max_length, padding='post', truncating='post'), corp_train], axis = 1)
labels_train = np.array(train_data['label'])

# valid
valid_sequences = np.concatenate([pad_sequences(tokenized_valid, maxlen=max_length, padding='post', truncating='post'), corp_valid], axis = 1)
labels_valid = np.array(validation_data['label'])

# test
test_sequences = np.concatenate([pad_sequences(tokenized_test, maxlen=max_length, padding='post', truncating='post'), corp_test], axis = 1)
labels_test = np.array(test_data['label'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Dense, Dropout, BatchNormalization
from focal_loss import SparseCategoricalFocalLoss
from tensorflow.keras.models import Model

# Define input layer
input_layer1 = Input(shape=(max_length+len(data['corp'].unique()),))
input_layer2 = Input(shape=(max_length+len(data['corp'].unique()),))

# Embedding layer
embedding_layer = Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=512, input_length=max_length)(input_layer)
embedding_layer = Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=512, input_length=max_length)(input_layer)

# LSTM layers
gru_1 = GRU(256, activation='tanh', return_sequences=True)(embedding_layer)
gru_2 = GRU(128, activation='tanh')(gru_1)

# Dropout and BatchNormalization layers
dropout_1 = Dropout(0.1)(gru_2)
batchnorm_1 = BatchNormalization()(dropout_1)
dense_1 = Dense(64, activation='relu')(batchnorm_1)

dropout_2 = Dropout(0.1)(dense_1)
batchnorm_2 = BatchNormalization()(dropout_2)
dense_2 = Dense(32, activation='relu')(batchnorm_2)

# Output layer
output_layer = Dense(9, activation='softmax')(dense_2)

# Create the functional model
model = Model(inputs=input_layer, outputs=output_layer)

# Create an optimizer with a custom learning rate
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Focal Loss
loss_func = SparseCategoricalFocalLoss(gamma=2)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_sequences,
                    labels_train,
                    validation_data=(valid_sequences, labels_valid),
                    verbose= 1,
                    epochs=100,
                    batch_size=64)

In [ ]:
predict_test = model.predict(test_sequences)

predicted_classes = np.argmax(predict_test, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# 예측 결과(predicted_labels)와 실제 레이블(true_labels) 비교
accuracy = accuracy_score(labels_test, predicted_classes)

f1_score = f1_score(labels_test, predicted_classes, average = 'weighted')


print(f'accuracy : {accuracy:.3f}\nF1-score : {f1_score:.3f}')

In [ ]:
from keras.models import load_model

# 모델 저장하기
model.save('./20230725_gru_best_up.h5')